# Lockbit 3.0 Ransomware Triage
> Taking a look at the new LockBit 3.0 Ransomware

- toc: true 
- badges: true
- categories: [Lockbit,lockbit3,yara,triage,ransomware]

## Overview

The sample: `80e8defa5377018b093b5b90de0f2957f7062144c83a09a56bba1fe4eda932ce` is on [MalwareBazaar](https://bazaar.abuse.ch/sample/80e8defa5377018b093b5b90de0f2957f7062144c83a09a56bba1fe4eda932ce/). The comments state that this needs to be run with a decryption key...

```
Execution method:

{04830965-76E6-6A9A-8EE1-6AF7499C1D08}.exe -k LocalServiceNetworkRestricted -pass db66023ab2abcb9957fb01ed50cdfa6a
```

### References 
- [Lockbit Black 3.0 Ransomware (infinitum IT) ](https://github.com/whichbuffer/Lockbit-Black-3.0/blob/main/Threat%20Spotlight%20Lockbit%20Black%203.0%20Ransomware.pdf)
- [LockBit 3.0: “Making The Ransomware Great Again” (custer25)](https://cluster25.io/2022/07/06/lockbit-3-0-making-the-ransomware-great-again/)


![](https://cdn.discordapp.com/attachments/885652495421030431/997227676190392351/THE_LOCKBIT_3.0_RANSOMWARE.png)


## Analysis


LOL! This is just a straight ripoff of BlackMatter... even our old gifs of us resolving the imports for BlackMatter match exactly the imports ... 

![](https://raw.githubusercontent.com/OALabs/hashdb-ida/main/assets/HashDB-IAT_Scan.gif)

## Config Extraction 

The config is stored similar to BlackMatter but it is in the `.data` section. 

The data is first decrypted with a hard-coded single byte XOR, the decompressed with aplib. Once it has been decompressed the data itself is a set of compiled shellcode at different offsets. The shellcode is then dynamically loaded and executed as part of the decryption routine.


### Helper Functions

In [2]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

### APLib 

Credit: [Sandor Nemes (snemes)](https://github.com/snemes/aplib/blob/master/aplib.py)

In [2]:
import struct
from binascii import crc32
from io import BytesIO

__all__ = ['APLib', 'decompress']
__version__ = '0.6'
__author__ = 'Sandor Nemes'


class APLib(object):

    __slots__ = 'source', 'destination', 'tag', 'bitcount', 'strict'

    def __init__(self, source, strict=True):
        self.source = BytesIO(source)
        self.destination = bytearray()
        self.tag = 0
        self.bitcount = 0
        self.strict = bool(strict)

    def getbit(self):
        # check if tag is empty
        self.bitcount -= 1
        if self.bitcount < 0:
            # load next tag
            self.tag = ord(self.source.read(1))
            self.bitcount = 7

        # shift bit out of tag
        bit = self.tag >> 7 & 1
        self.tag <<= 1

        return bit

    def getgamma(self):
        result = 1

        # input gamma2-encoded bits
        while True:
            result = (result << 1) + self.getbit()
            if not self.getbit():
                break

        return result

    def depack(self):
        r0 = -1
        lwm = 0
        done = False

        try:

            # first byte verbatim
            self.destination += self.source.read(1)

            # main decompression loop
            while not done:
                if self.getbit():
                    if self.getbit():
                        if self.getbit():
                            offs = 0
                            for _ in range(4):
                                offs = (offs << 1) + self.getbit()

                            if offs:
                                self.destination.append(self.destination[-offs])
                            else:
                                self.destination.append(0)

                            lwm = 0
                        else:
                            offs = ord(self.source.read(1))
                            length = 2 + (offs & 1)
                            offs >>= 1

                            if offs:
                                for _ in range(length):
                                    self.destination.append(self.destination[-offs])
                            else:
                                done = True

                            r0 = offs
                            lwm = 1
                    else:
                        offs = self.getgamma()

                        if lwm == 0 and offs == 2:
                            offs = r0
                            length = self.getgamma()

                            for _ in range(length):
                                self.destination.append(self.destination[-offs])
                        else:
                            if lwm == 0:
                                offs -= 3
                            else:
                                offs -= 2

                            offs <<= 8
                            offs += ord(self.source.read(1))
                            length = self.getgamma()

                            if offs >= 32000:
                                length += 1
                            if offs >= 1280:
                                length += 1
                            if offs < 128:
                                length += 2

                            for _ in range(length):
                                self.destination.append(self.destination[-offs])

                            r0 = offs

                        lwm = 1
                else:
                    self.destination += self.source.read(1)
                    lwm = 0

        except (TypeError, IndexError):
            if self.strict:
                raise RuntimeError('aPLib decompression error')

        return bytes(self.destination)

    def pack(self):
        raise NotImplementedError


def aplib_decompress(data, strict=False):
    packed_size = None
    packed_crc = None
    orig_size = None
    orig_crc = None
    if data.startswith(b'AP32') and len(data) >= 24:
        # data has an aPLib header
        header_size, packed_size, packed_crc, orig_size, orig_crc = struct.unpack_from('=IIIII', data, 4)
        data = data[header_size : header_size + packed_size]
    if strict:
        if packed_size is not None and packed_size != len(data):
            raise RuntimeError('Packed data size is incorrect')
        if packed_crc is not None and packed_crc != crc32(data):
            raise RuntimeError('Packed data checksum is incorrect')
    result = APLib(data, strict=strict).depack()
    if strict:
        if orig_size is not None and orig_size != len(result):
            raise RuntimeError('Unpacked data size is incorrect')
        if orig_crc is not None and orig_crc != crc32(result):
            raise RuntimeError('Unpacked data checksum is incorrect')
    return result

In [3]:
def xor_decrypt(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i%len(key)])
    return bytes(out)


def xor_byte_decrypt(data, key_byte):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key_byte)
    return bytes(out) 

### Config PRNG Code

The code to generate the keys used to decrypt the config is itself encrypted and compressed. At runtime it is decrypted and loaded into memory. 

In [4]:
config_data = unhex('6330615a31683f92c7f1d1ae327037a5f0b4134439c9f7c0b036c2696bf35a372b6803f90bfc3e342082c834ca6e7001bbf8f1f93dfe3fe0def2cf0dd7176581dc6163cc753cca7d28243bf83a2045ddb02a38c7d16b696d4cf2d18e2098e83f25552433384fdee39429eb5d5afc3f332df034581dc46168f03a4fa57c626028d89dcf32357f2cb157c7f7e24e4ba124b933fb56726238040ccf43432903d8be0f6ba1b238ee3dd8001327159516363a7338264b3c732b10204b24db3c28b12cdd353c10b114dccaceb470181cc6dfb6700004c6d4b670080cc6e9b6707074c6feb670787cc6f3b6706064c688b670686c9283143b503f54589254583f5c58a754403f4458bc5148b1d267b13c124c4b88a2493759e235b4382172c7d2120c521c342aa4e70f66678c45c7614df6c2b707361366349b378b679d4700d5772fa7d072071876df07663c12580b6210f878109c7227297620702437580b73f86010f1c2cdb87628f8702c0b71ea2e716832e6e794101038140b71ea2e715032db87941018381c0b71ea2e71586430781000200446b3840c40b2b52547781008200c46b3840c48b04c6f796e3f5030300000')

ptxt_config_data = xor_byte_decrypt(config_data, 0x30)

dec_config_data = aplib_decompress(ptxt_config_data)

open('/tmp/lb_dec.bin','wb').write(dec_config_data)



681

### Config Layout and Data

There are three components to the config each that seem to use a different shellcode PRNG.... the "config" which uses the same PRNG key as BlackMatter, the ransome note, and the salsa matri, both of which use a different PRNG key.

#### Config (BlackMatter PRNG Key)

In [48]:
import struct 

def is_ascii(s):
    return all(c < 128 or c == 0 for c in s)

def gen_key(seed, key_length):
    out = b''
    tmp_seed = seed
    for i in range(key_length):
        tmp_seed = (0x5851F42D4C957F2D * tmp_seed + 0x14057B7EF767814F) & 0xFFFFFFFFFFFFFFFF
        out += struct.pack('<Q',(tmp_seed * seed)& 0xFFFFFFFFFFFFFFFF)
    return out


def decrypt(enc_data, seed):
    out = []
    padding  = 8 - len(enc_data) % 8
    if padding != 8:
        enc_data += b'\x00' * padding
        
    data_size = len(enc_data)
    
    keystream = gen_key(seed, data_size)
    for i in range(0,data_size,8):
        out.append(enc_data[i] ^ keystream[i])
        out.append(enc_data[i+1] ^ keystream[i+5])
        out.append(enc_data[i+2] ^ keystream[i+1])
        out.append(enc_data[i+3] ^ keystream[i+4])
        out.append(enc_data[i+4] ^ keystream[i+2])
        out.append(enc_data[i+5] ^ keystream[i+7])
        out.append(enc_data[i+6] ^ keystream[i+3])
        out.append(enc_data[i+7] ^ keystream[i+6])
    if padding != 8:
        out = out[:-padding]
    return bytes(out)


import requests

def resolve_hash(hash_value):
    out = requests.get('https://hashdb.openanalysis.net/hash/add_ror13/' + str(hash_value))
    if out.ok:
        hashes = out.json().get('hashes',[])
        if len(hashes) != 0:
            return hashes[0].get('string',{}).get('string','')
        else:
            return ''
    else:
        return ''




In [61]:
config_data = unhex('...')

seed = struct.unpack('<Q', config_data[:8])[0]
data_size = struct.unpack('<I',config_data[8:12])[0]
enc_data = config_data[12:12+data_size]

ptxt_data = aplib_decompress(decrypt(enc_data, seed))

ptxt_data


b'\xb3\x0c\x86"\xdbS\xffY]\xc0Vm(\xcd\xb9\xec\xcd\x1e\xe6\xaf\xb4\xa6\xc5z\xa1\x02\xa7\xa9)W\x8c\xfb\xb5AV\xc4\xf4\x98\x06\x93\xe0D\xa0\x04;\xb6\x80*\x8b\x13\xdd\xfb7\xb2\x84K\xa3\xadS\x94\x8b\x0egit\x84\xcb\x1f\xe2\x02\x06\xe4D\tY\x04\xcc\x91\xfa\xe53]\xa7\xe8\xb8Os\x13\xd0x\xa9ZQ\xbaLJV\x02\x93^A\xb6\xef%$]H\x8f\xa7\x7ftE\xa2XP%\xafgC\x1b\xf7~\xbc:L\xa4\x08r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x01\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01\x00\x01\x01\x01\x01\x00\x00\x00\x01\x00\x00(\x00\x00\x00\xa9\x00\x00\x00\xea\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfb\x01\x00\x00\xd0\x04\x00\x00\x00\x00\x00\x00\xc5\x05\x00\x00\x02\x06\x00\x00LSEKA82B8oz1eHAmNX5oJtdsQuNYac7Gp9HypsA7puE4C8tS3bjK3E5OADaVZQirlON1Lq6OAa7UJUtMNXnwB3X5ZmuSOOq3lOZmU5tYG817Ot5cO2IiurM3Ou8AAAAA\x00FarMhpsJBzmWrgzwVqvI/FKi0oIA7GuEN1mX2/WmOsLkV6qFNariy9H3zsgAAAAA\x00AA6wZwAZsMWAG0jFQBigx0AacMdAGBDJQBsgycAbaMkAHGDJQB2Qyf7L/

In [7]:
import base64
import time

ptr = 0
rsa_data = ptxt_data[ptr:128]
ptr += 128
affiliate_id_data = ptxt_data[ptr:ptr+32]
ptr+= 32
config_flags = ptxt_data[ptr:ptr+24]
ptr+= 24

b64_strings_offset = struct.unpack('<I',ptxt_data[ptr:ptr+4])[0] + ptr +4

for bstr in ptxt_data[b64_strings_offset:].split(b'\x00'):
    try:
        config_value = base64.b64decode(bstr)
        if is_ascii(config_value):
            print("string list")
            for s in config_value.split(b'\x00\x00'):
                ss = s.replace(b'\x00',b'')
                print(f"\t{ss}")
        else:
            print("hash list")
            print(config_value)
            for i in range(0,len(config_value),4):
                time.sleep(1)
                hash_value = struct.unpack('<I',config_value[i:i+4])[0]
                hash_string = resolve_hash(hash_value)
                print(f"\t{hex(hash_value)} : {hash_string}")
    except:
        pass



hash list
b'\x03\xcd\x81\xf2\x8c\xf5xp&5~h&\xd7lB\xe3Xi\xce\xc6\xa7\xd1\xf2\xa6\xc0;\xa6\xe18\x0b\xcbR\xdd\xb8\xca\xdcNN\x006\x95e\x08\xab\x94\xe3u.\xae\x8e\x01\xae\xd4%KL5y\xf0\x07u\xf9fk\x928\xea\xb7\x94\xe6fS\x9bX\x1b\xcd{:\xde\\;b"\xba\xb37:\xef\x00\x00\x00\x00'
	0xf281cd03 : 
	0x7078f58c : 
	0x687e3526 : 
	0x426cd726 : 
	0xce6958e3 : 
	0xf2d1a7c6 : 
	0xa63bc0a6 : 
	0xcb0b38e1 : 
	0xcab8dd52 : 
	0x4e4edc : 
	0x8659536 : 
	0x75e394ab : 
	0x18eae2e : 
	0x4b25d4ae : 
	0xf079354c : 
	0x66f97507 : 
	0xea38926b : 
	0x66e694b7 : 
hash list
b'\x15\xaa\xcc\x86\x9b\t\x079\x96\xae\x0c\xf0V\xab\xc8\xfcR\xa2\xd2\x82\x00\xeck\x847Y\x97\xdb\xf5\xa6:\xc2\xe4W\xaa\x855\xaa\xe2\xcb\xd1\xf7\xce\xc8\x00\x00\x00\x00'
	0x86ccaa15 : autorun.inf
	0x3907099b : boot.ini
	0xf00cae96 : bootfont.bin
	0xfcc8ab56 : bootsect.bak
hash list
b'\x00\x0e\xb0g\x00\x19\xb0\xc5\x80\x1bH\xc5@\x18\xa0\xc7@\x1ap\xc7@\x18\x10\xc9@\x1b \xc9\xc0\x1bh\xc9\x00\x1c`\xc9@\x1d\x90\xc9\xfe\xcb\xfc\xa1\xf1\x94\xbaJ1\x96\xe2Jq\x97\xe2

### Encrypted Strings
Along witht the config the `.data` section contains encrypted data blobs which are preceded by a DWORD containing their length. The blobs use the same seed and PRNG as the config so once that has been extracted from the config each blob can be decrypted. Most of the blobs are strings, but a few of them are data. Some of the larger strings are also compressed using aplib.


In [60]:
data = unhex('d8e23d69318921fe8a2b6e115e5a35fbaeb1c1ac5ccfc4542be7b3931b8d7af40ebc084a3a22c85058e6907d2286fb5cae0889a99b9c37b100eb7877fc6acda615b2a8b014320d2e4e42ef5bfc427e3cbf60a4e8b215df27c51283c5121ef1d58b0756a9690da293f87fecffc9cf77f99ce44b6e4eed1325a204ebeea91aa9bd310072129baa420e8297e9b72cb704e58c35fa63421ab9b5748c35b8d005cdb6aacfac9f22838aca4b5dbecfa66d35e9f093caa18a8acaab51f334b73ee1fbca5202ad2e499a213f8706716cf55784b4e4c903b05d337fac0e73e1648899e42dd12311290d2bdf65f775f4a56336eaae5e010000')

#aplib_decompress(decrypt(enc_data, data_size, seed))

decrypt(data, seed).replace(b'\x00',b'')


b'LockBit Black\r\n\r\nAll your important files are stolen and encrypted!\r\nYou must find %s file\r\nand follow the instruction!"\x04\xac'

In [57]:
import pefile
import struct
import hashlib

file_data = open('/tmp/lock3_dump_SCY.bin', 'rb').read()

pe = pefile.PE(data=file_data)

section_data = None

for s in pe.sections:
    if b'.data\x00' == s.Name[:6]:
        section_data = s.get_data()

        
ptr = 0 
enc_strings = []

while ptr < len(section_data):
    blob_size = struct.unpack('<I', section_data[ptr:ptr+4])[0]
    if blob_size == 0:
        break
    
    enc_strings.append(section_data[ptr+4:ptr+4+blob_size])
    ptr = ptr+4+blob_size

    
ico_magic = b'\x00\x00\x01\x00\x03'
  
for e in enc_strings:
    print("\n")
    ptxt_blob = decrypt(e, seed)
    if is_ascii(ptxt_blob):
        print(ptxt_blob.replace(b'\x00',b''))
    else:
        dec_blob = aplib_decompress(ptxt_blob)
        if len(dec_blob) <= len(ptxt_blob):
            print(f"** Raw data: {hashlib.sha256(ptxt_blob).hexdigest()}")
            tmp_filename = "/tmp/" + hashlib.sha256(ptxt_blob).hexdigest() + '.bin'
            open(tmp_filename, 'wb').write(ptxt_blob)
        elif is_ascii(dec_blob):
            print(dec_blob.replace(b'\x00',b''))
        elif ico_magic == dec_blob[:len(ico_magic)]:
            print("** .ico file")
            tmp_filename = "/tmp/" + hashlib.sha256(dec_blob).hexdigest() + '_ico.bin'
            open(tmp_filename, 'wb').write(dec_blob)
        elif dec_blob[:2] == b'MZ':
            print("** PE file")
            tmp_filename = "/tmp/" + hashlib.sha256(dec_blob).hexdigest() + '_pe.bin'
            open(tmp_filename, 'wb').write(dec_blob)
        else:
            print("WTF??\n")
            print(dec_blob)
            
            
            




b'LockBit Black\r\n\r\nAll your important files are stolen and encrypted!\r\nYou must find %s file\r\nand follow the instruction!'


b'                               LockBit Black\r\n\r\nAll your important files are stolen and encrypted!\r\n    You must find %s file\r\n                    and follow the instruction!'


** Raw data: da2d72e132f9e8706676a72d9bf48a2bd26d9f8014686424ddb927b87637f2a1


** Raw data: 2df9218c31586e1ce85d989ea97d9e6a6664d4a50b681914a5b6c44064d5c146


** Raw data: d8dd2530a4b2165f16547c237e43dc15adb6e168af5ae8b153793d7aee863d93


b'LockBit Black Ransomware\r\n\r\nYour data are stolen and encrypted\r\n\r\nThe data will be published on TOR website\r\nhttp://lockbitapt2yfbt7lchxejug47kmqvqqxvvjpqkmevv4l3azl3gy6pyd.onion\r\nand http://lockbitapt.uz if you do not pay the ransom\r\n\r\nYou can contact us and decrypt one file for free on these TOR sites\r\nhttp://lockbitsupa7e3b4pkn4mgkgojrl5iqgx24clbzc4xm7i6jeetsia3qd.onion\r\nhttp://lockbitsupn2h6be2cnqpvncyhj4rgm

In [25]:
len(b'LockBit Black\r\n\r\nAll your important files are stolen and encrypted!\r\nYou must find %s file\r\nand follow the instructi')

116

2000

In [41]:
data = '1'
8 - len(data) % 8 

7

In [52]:
open('/tmp/something.bin','wb').write(b'\x00\x00\x01\x00\x03\x0000\x00\x00\x01\x00 \x00\xa8%\x00\x006\x00\x00\x00  \x00\x00\x01\x00 \x00\xa8\x10\x00\x00\xde%\x00\x00\x10\x10\x00\x00\x01\x00 \x00h\x04\x00\x00\x866\x00\x00(\x00\x00\x000\x00\x00\x00`\x00\x00\x00\x01\x00 \x00\x00\x00\x00\x00\x80%\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xaa\xaa\xaa\x00\x00\x00\x00\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\xc3\xc3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x03\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xd0\xd0\xd0\x00\xff\xff\xffP\xff\xff\xff\x85\xff\xff\xff\x86\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x86\xff\xff\xff\x81\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x84\xff\xff\xff\x83\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x87\xff\xff\xff\x86\xff\xff\xff\x86\xff\xff\xffb\x00\x00\x00\x00\xf9\xf9\xf9\x08\xe4\xe4\xe0\x19\xcd\xcd\xcd\x0c\xa8\xa8\xa8\x00\xfb\xfb\xfb\x00\xfd\xfd\xfd\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\xed\xed\xed\x1a\xff\xff\xff\xfc\xcf\xcf\xcf\xff\xc2\xc2\xc3\xff\xc5\xc5\xc6\xff\xc6\xc6\xc6\xff\xc6\xc5\xc6\xff\xc6\xc5\xc6\xff\xc6\xc5\xc6\xff\xc6\xc5\xc6\xff\xc6\xc5\xc6\xff\xc6\xc5\xc6\xff\xc6\xc5\xc6\xff\xc6\xc5\xc6\xff\xc6\xc5\xc6\xff\xc6\xc6\xc7\xff\xc2\xc1\xc2\xff\xc6\xc6\xc6\xff\xed\xed\xed\xff\xc3\xc3\xc3\xff\xc4\xc4\xc5\xff\xc4\xc3\xc4\xff\xc6\xc6\xc7\xff\xb9\xb9\xb9\xff\xdf\xdf\xe0\xff\xdb\xdb\xdb\xff\xc0\xc0\xc1\xff\xc4\xc4\xc5\xff\xc5\xc5\xc6\xff\xc3\xc3\xc4\xff\xc4\xc4\xc5\xff\xfd\xfd\xfd\xff\xfd\xfd\xfdp\xff\xff\xff\xd4\xff\xff\xff\xf1\xff\xff\xff\xe1\xff\xff\xff\xa6\xff\xff\xff]\xf7\xf7\xf7\x05\xfe\xfe\xfe\x00\xff\xff\xff\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xff\xff\xff&\xff\xff\xff\xf8+*+\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfbyy{\xfb===\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb##\'\xfb\x82\x82\x88\xfb\r\r\r\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x94\x94\x95\xfb\xfb\xfb\xfb\xfa\xff\xff\xff\xff\xbc\xbc\xbd\xffONS\xff\x82\x82\x88\xff\xc0\xc0\xc0\xff\xfb\xfa\xfb\xff\xff\xff\xff\xcc\xfc\xfc\xfcT\xfe\xfe\xfe\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\xf9\xf9\xf9$\xff\xff\xff\xf9554\xff\x02\x02\x02\xfe\x0e\x0e\x0e\xff))*\xff*)*\xff**+\xff**+\xff**+\xff**+\xff**+\xff**+\xff**+\xff**+\xff//0\xff\x83\x82\x84\xffIIJ\xff\x00\x00\x00\xff\x02\x02\x02\xff\x04\x04\x04\xff\x00\x00\x00\xff//3\xff\x90\x90\x97\xff\x15\x15\x16\xff\x00\x00\x00\xff\x04\x04\x04\xff\x02\x02\x01\xff\x05\x05\x05\xff\x9f\x9f\xa0\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xdb\xdb\xdb\xfd\x1e\x1e\x1e\xfc\x00\x00\x00\xfe\x00\x00\x00\xfd\x00\x00\x00\xfb88<\xfa\xac\xab\xb0\xff\xff\xff\xff\xff\xfb\xfb\xfc\x81\xfc\xfc\xfc\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa666\xff\x00\x00\x00\xffccd\xff\x87\x87\x88\xffeef\xffkkl\xfflll\xfflll\xfflll\xfflll\xfflll\xffkkl\xffjjj\xffqqr\xff;;;\xff\x00\x00\x00\xff\x04\x04\x04\xff\x03\x03\x03\xff\x00\x00\x00\xff//3\xff\x90\x90\x98\xff\x13\x13\x14\xff\x00\x00\x00\xff\x04\x04\x04\xfe\x00\x00\x00\xff\n\n\n\xfd\xb4\xb4\xb5\xfd\xff\xff\xff\xff\xff\xff\xff\xfe\xc5\xc5\xc5\xfejjj\xfeuuw\xff~~\x80\xfeccc\xff\x17\x17\x17\xfe\x00\x00\x00\xff\x00\x00\x00\xfcqqq\xfa\xfe\xfe\xfe\xff\xf7\xf7\xf7u\xfd\xfd\xfd\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa777\xff\x00\x00\x00\xffggh\xff&&&\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff(\'*\xff\x87\x86\x8e\xff\n\n\n\xff\x00\x00\x00\xff\x00\x00\x00\xfe\x00\x00\x00\xff\x07\x07\x07\xfd\xba\xb9\xba\xff\xff\xff\xff\xff\xfd\xfd\xfd\xff\xbb\xbb\xbb\xff\x06\x06\x06\xff\x00\x00\x00\xff\x00\x00\x00\xff\x12\x12\x13\xffEEE\xff\x88\x88\x8c\xffEEM\xfe\x00\x00\x00\xff\x00\x00\x00\xfd\x8a\x8a\x8c\xfa\xff\xff\xff\xf2\xeb\xeb\xeb!\x00\x00\x00\x00\xff\xff\xff\x01\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa666\xff\x00\x00\x00\xffffg\xff///\xff\x04\x04\x04\xfe\x1f\x1f \xff***\xfe+++\xfd+++\xfd)))\xfd)))\xfd)))\xfd)))\xfd)))\xfd(((\xfd\'\'\'\xfd###\xfd>=@\xfd\xa5\xa4\xae\xfc444\xfd"""\xfd\'\'\'\xfd%%%\xfd+++\xfc\xb9\xb9\xb9\xff\xff\xff\xff\xe4\xfb\xfb\xfb\xe0\xd7\xd6\xd9\xfc323\xfc+++\xfd"""\xfb\x00\x00\x00\xfc\x00\x00\x00\xfe\x00\x00\x00\xfe\x07\x07\t\xfe\x86\x84\x96\xff88=\xfe\x00\x00\x00\xff\x04\x04\t\xfa\xe0\xe0\xe3\xff\xfe\xfe\xfe\x8c\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa777\xff\x00\x00\x00\xffkjk\xffBBB\xff\x00\x00\x00\xff\xb3\xb3\xb3\xff\xf9\xf9\xf9\xff\xf4\xf4\xf4\xff\xf7\xf7\xf7\xff\xf8\xf8\xf8\xff\xf8\xf8\xf8\xff\xf8\xf8\xf8\xff\xf8\xf8\xf8\xff\xf8\xf8\xf8\xff\xf8\xf8\xf8\xff\xf8\xf8\xf8\xff\xf6\xf6\xf6\xff\xff\xfe\xff\xff\xf9\xf9\xf9\xff\xf3\xf3\xf3\xff\xf9\xf9\xf9\xff\xf8\xf8\xf8\xff\xf7\xf7\xf7\xff\xfa\xfa\xfa\xff\xff\xff\xff\xee\xfb\xfb\xfb.\xff\xff\xff\x9b\xf5\xf5\xf6\xff\xf7\xf7\xf7\xff\xfb\xfb\xfb\xff\xf0\xf0\xf0\xff\xbf\xbf\xbf\xffGFG\xfd\x00\x00\x00\xff\x00\x00\x00\xff\r\r\x0e\xff\x85\x84\x8a\xff\x08\x08\x08\xfe\x00\x00\x00\xfd\x90\x90\x90\xfe\xff\xff\xff\xda\xb9\xb9\xb9\x07\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa333\xff\x00\x00\x00\xff\x19\x19\x19\xff\x8f\x8e\x8f\xff\x1e\x1e\x1f\xfa\xae\xae\xae\xff\xff\xff\xff\xaa\xff\xff\xff0\xff\xff\xff5\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xff\xff\xff4\xff\xff\xffA\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xff\xff\xff1\xf4\xf4\xf4\x18\x00\x00\x00\x00\xfe\xfe\xfe\x11\xff\xff\xffB\xff\xff\xff1\xff\xff\xff1\xff\xff\xffP\xff\xff\xff\xbb\xff\xff\xff\xffXXX\xfd\x00\x00\x00\xfe\x00\x00\x00\xfeget\xff;;?\xff\x00\x00\x00\xfeXWZ\xff\xff\xff\xff\xee\xc8\xc8\xc6\x12\x00\x00\x00\x00\xff\xff\xff\x01\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa222\xff\x00\x00\x00\xff\x00\x00\x00\xff\x1a\x1a\x1a\xff}}~\xf9\xd2\xd2\xd2\xff\xfd\xfd\xfd\x91\xfe\xfe\xfe\x00\xfe\xfe\xfe\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfb\xfb\xfb\x00\xfe\xfe\xfe\x00\xf8\xf7\xf8\x00\xff\xff\xff\x9c\xe7\xe8\xe8\xff\x12\x12\x13\xfb\x00\x00\x00\xffBBH\xff[Z_\xff\x00\x00\x00\xfeBBC\xfd\xff\xff\xff\xfc\xf6\xf6\xf50\x00\x00\x00\x00\xff\xff\xff\x04\xff\xff\xff\x04\x00\x00\x00\x00\xfa\xfa\xfa$\xff\xff\xff\xf9444\xff\x00\x00\x00\xfe\x04\x04\x04\xff\x00\x00\x00\xff\x0f\x0f\x0f\xfa\xd9\xd9\xd9\xff\xfe\xfe\xfe\x93\xfe\xfe\xfe\x02\xff\xff\xff\x06\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\xff\xff\xff\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\xff\xff\xff\x02\xff\xff\xff\x02\xfe\xfe\xfe\x04\xff\xff\xff\x01\xed\xed\xed&\xff\xff\xff\xfbMMN\xfd\x00\x00\x00\xfe553\xffeee\xff\x00\x00\x00\xff===\xfc\xff\xff\xff\xff\xfa\xfa\xfa:\x00\x00\x00\x00\xff\xff\xff\x05\xff\xff\xff\x04\x00\x00\x00\x00\xfc\xfc\xfc%\xff\xff\xff\xfa***\xff\x01\x01\x01\xff\x01\x01\x01\xff\x03\x03\x03\xff\x00\x00\x00\xfa\xb2\xb2\xb2\xff\xff\xff\xff\x96\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfe\xfe\xfe\x00\x00\x00\x00\x00\xef\xef\xef$\xff\xff\xff\xfcHHH\xfd\x00\x00\x00\xfe:9<\xffccf\xff\x00\x00\x00\xfeHHI\xfe\xff\xff\xff\xf5\xec\xec\xeb\x1f\x00\x00\x00\x00\xff\xff\xff\x02\xff\xff\xff\x04\x00\x00\x00\x00\xee\xee\xee"\xff\xff\xff\xf8^^^\xff\x00\x00\x00\xfe\x04\x04\x04\xff\x01\x01\x01\xff\x00\x00\x00\xfa\xb8\xb8\xb8\xff\xff\xff\xff\x95\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00III\x00\x00\x00\x00\x04\x14\x14\x14\x04\x11\x11\x11\x04\x11\x11\x11\x04\x11\x11\x11\x04\x11\x11\x11\x04\x11\x11\x11\x04\x11\x11\x11\x04\x12\x12\x11\x04\xed\xed\xed\x04\xfa\xfa\xfa\x06\xf6\xf5\xf5 \xff\xff\xff\xb0\xe6\xe6\xe7\xfe\x11\x11\x11\xfc\x00\x00\x00\xff[Z^\xffHHM\xff\x00\x00\x00\xfe^^_\xff\xff\xff\xff\xea\xc4\xc4\xc3\x10\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xf7\xf7\xf7#\xf5\xf5\xf5\xf9\xa4\xa3\xa4\xffJJJ\xfe\x00\x00\x00\xff\x05\x05\x05\xff\x00\x00\x00\xfa\xb8\xb8\xb8\xff\xff\xff\xff\x96\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\xfc\xfc\xfc1\xff\xff\xff\xc5\xff\xff\xff\xd9\xff\xff\xff\xda\xff\xff\xff\xdb\xff\xff\xff\xdb\xff\xff\xff\xdb\xff\xff\xff\xdb\xff\xff\xff\xdb\xff\xff\xff\xdb\xff\xff\xff\xdb\xff\xff\xff\xdc\xff\xff\xff\xdb\xff\xff\xff\xf4\xe9\xe9\xeb\xffKKL\xfe\x00\x00\x00\xff\x04\x05\x03\xfe\x87\x87\x87\xff\x10\x10\x0f\xfe\x00\x00\x00\xfd\x83\x83\x83\xfe\xff\xff\xff\xdf\xb7\xb7\xb7\t\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfd\xfd\xfd%\xfd\xfd\xfd\xfa))*\xffoop\xffTTT\xff\x00\x00\x00\xff\x01\x01\x01\xfa\xb7\xb7\xb7\xff\xff\xff\xff\x96\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\xfa\xfa\xfa6\xff\xff\xff\xe9\xce\xce\xce\xff\x96\x96\x96\xff\x9a\x9a\x9a\xff\x99\x99\x99\xff\x99\x99\x99\xff\x9a\x9a\x9a\xff\x98\x98\x98\xff\x97\x97\x97\xff\x96\x96\x96\xff\x96\x96\x96\xff\x93\x93\x93\xff\x8d\x8d\x8f\xfffeh\xff\x1a\x19\x1c\xfd\x00\x00\x00\xff\x00\x00\x00\xfe{y\x8a\xffCCE\xff\x00\x00\x00\xff\x00\x00\x00\xfb\xc5\xc5\xc6\xfe\xff\xff\xff\xa8\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfa\xfa\xfa$\xff\xff\xff\xf9222\xff\x00\x00\x00\xfe}}~\xffTTT\xff\x00\x00\x00\xfa\xba\xba\xba\xff\xff\xff\xff\x95\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x01\xfe\xfe\xfe\x00\xf8\xf8\xf8?\xfd\xfd\xfd\xf1\xb5\xb4\xb5\xff\x00\x00\x00\xfb\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfe\x00\x00\x00\xff)(,\xfe\x84\x83\x8c\xffWUe\xff\x00\x00\x00\xff\x00\x00\x00\xff889\xfb\xfb\xfb\xfb\xff\xf1\xf1\xf1T\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa444\xff\x00\x00\x00\xff\x00\x00\x00\xff}}~\xffFFF\xfa\xac\xac\xac\xff\xff\xff\xff\x96\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x01\x00\x00\x00\x00\xf6\xf6\xf69\xff\xff\xff\xee\xed\xed\xee\xffTTU\xfcHHI\xffPOP\xfeONO\xffOOO\xffMMN\xffSRS\xff\x1a\x1a\x1a\xff\x00\x00\x00\xff\x05\x05\x05\xff\x02\x02\x02\xff\x19\x18\x19\xff445\xffVVV\xffzzz\xffpox\xff\'\'+\xff\x00\x00\x00\xff\x02\x02\x02\xff===\xfb\xe0\xe0\xe0\xff\xfd\xfd\xfd\xb6\xef\xef\xf0\x01\xff\xff\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb$\xff\xff\xff\xfa222\xff\x00\x00\x00\xff\x03\x03\x03\xff\x00\x00\x00\xffjjj\xfa\xe4\xe4\xe4\xff\xfd\xfd\xfd\x93\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xeb\xeb\xeb\x0e\xfe\xfe\xfe\xea\xc5\xc5\xc5\xffEEE\xfcOOO\xffSRS\xfeQQQ\xffRRR\xffRQR\xffPPQ\xffYYY\xff\x90\x90\x90\xff(((\xff\x00\x00\x00\xff\x04\x04\x04\xff??@\xff\x81\x80\x83\xff\xa1\xa1\xa1\xff\xa0\xa0\xa1\xffddd\xff\x1e\x1e\x1e\xff\x00\x00\x00\xff\x01\x01\x01\xffDDE\xfa\xea\xea\xeb\xff\xfc\xfc\xfc\xa5\xf5\xf5\xf5\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa333\xff\x00\x00\x00\xff\x02\x02\x02\xff\x00\x00\x00\xff\x00\x00\x00\xfa\xc6\xc6\xc6\xff\xfe\xfe\xfe\x94\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x01\x00\x00\x00\x00\xf5\xf5\xf4 \xff\xff\xff\xfb323\xfc\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\t\t\t\xfd\x81\x81\x81\xfd\x1a\x1a\x1a\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfe\x00\x00\x00\xff\x00\x00\x00\xfe888\xff\x82\x82\x82\xffYY[\xff\x00\x00\x00\xff\x00\x00\x00\xfeHHI\xfb\xfe\xfe\xfe\xff\xf0\xf0\xf0J\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xf8\xf8\xf8$\xff\xff\xff\xf9...\xff\x00\x00\x00\xfe\x02\x02\x03\xff\x02\x02\x02\xff\x00\x00\x00\xfa\xb5\xb5\xb4\xff\xff\xff\xff\x96\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfc\xfc\xfc\x1d\xff\xff\xff\xff\x8a\x8a\x8a\xff^^^\xffccc\xffaaa\xffaaa\xffaaa\xffaaa\xffaaa\xffbbb\xffZZZ\xffvvv\xff\xbc\xbc\xbc\xffaaa\xff[[]\xffRRS\xff\'\'\'\xfc\x00\x00\x00\xfe\x00\x00\x00\xff\x00\x00\x00\xfeuty\xffIIM\xff\x00\x00\x00\xff\x00\x00\x00\xfa\xd0\xd0\xd0\xfe\xff\xff\xff\x9d\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xed\xed\xed"\xfa\xfa\xfa\xf8\x9e\x9e\xa1\xff\x0b\x0b\x0c\xfe\x00\x00\x00\xff\x03\x03\x03\xff\x00\x00\x00\xfa\xb8\xb8\xb8\xff\xff\xff\xff\x95\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x05\xff\xff\xff\xa7\xff\xff\xff\xdd\xff\xff\xff\xdc\xff\xff\xff\xde\xff\xff\xff\xde\xff\xff\xff\xde\xff\xff\xff\xde\xff\xff\xff\xde\xff\xff\xff\xde\xff\xff\xff\xde\xff\xff\xff\xde\xff\xff\xff\xde\xff\xff\xff\xdb\xff\xff\xff\xdd\xff\xff\xff\xdf\xff\xff\xff\xf5\xf6\xf6\xf6\xff\xa2\xa2\xa2\xff\x0e\x0e\x0e\xfe\x00\x00\x00\xff\x02\x02\x05\xff\x87\x85\x8f\xff\x0e\x0e\x0f\xfe\x00\x00\x00\xfc\x8e\x8f\x8e\xfe\xff\xff\xff\xd6\xcd\xcd\xce\x05\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfe\xfe\xfe%\xf7\xf7\xf7\xfadde\xff\x81\x81\x85\xff\x10\x10\x11\xff\x00\x00\x00\xff\x00\x00\x00\xfa\xb7\xb7\xb7\xff\xff\xff\xff\x96\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x85\x85\x85\x05uuu\x06uuu\x06sss\x06sss\x06sss\x06sss\x06sss\x06uuu\x06sss\x06\xa3\xa3\xa3\x06\x94\x93\x93\x04\xdd\xdd\xdd\x06\xf8\xf8\xf8\x08\xfb\xfb\xfb\x1f\xf7\xf7\xf7j\xff\xff\xff\xeb\xa9\xa9\xaa\xfd\x00\x00\x00\xfd\x00\x00\x00\xff___\xff??@\xff\x00\x00\x00\xfdddi\xfe\xff\xff\xff\xe9\xb8\xb9\xb5\x0f\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfa\xfa\xfa$\xff\xff\xff\xf9""!\xff+,,\xfe\x87\x86\x8a\xff\x0e\x0e\x0e\xff\x00\x00\x00\xfa\xb8\xb8\xb8\xff\xff\xff\xff\x96\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfe\xfe\xfe\x00\xf9\xf9\xf9X\xff\xff\xff\xff---\xfb\x00\x00\x00\xff@@A\xff[[\\\xff\x00\x00\x00\xfeMLT\xff\xff\xff\xff\xf1\xe4\xe5\xde\x19\x00\x00\x00\x00\xff\xff\xff\x01\xff\xff\xff\x04\x00\x00\x00\x00\xfa\xfa\xfa%\xff\xff\xff\xfa999\xff\x00\x00\x00\xffmmn\xff444\xff\x00\x00\x00\xfa\xb9\xb9\xb9\xff\xff\xff\xff\x95\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\xfd\xfd\x02\xfe\xfe\xfe\x06\xe6\xe6\xe6\x1c\xfd\xfd\xfd\xf8TTT\xfe\x00\x00\x00\xfd655\xffeee\xff\x00\x00\x00\xff??>\xfc\xff\xff\xff\xfe\xf9\xf9\xf95\x00\x00\x00\x00\xff\xff\xff\x04\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa666\xff\x01\x01\x01\xffddf\xff333\xff\x01\x01\x01\xf9\xb6\xb6\xb6\xff\xff\xff\xff\x95\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf1\xf1\xf1.\xff\xff\xff\xfdDDD\xfc\x00\x00\x00\xfe::=\xffaab\xff\x00\x00\x00\xfe>>?\xfc\xff\xff\xff\xfe\xf9\xf9\xf97\x00\x00\x00\x00\xff\xff\xff\x04\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa777\xff\x00\x00\x00\xffihi\xff...\xff\x00\x00\x00\xfa\xca\xca\xca\xff\xfc\xfc\xfc\x96\xfc\xfc\xfc\x07\xfc\xfc\xfc\x0c\xb4\xb4\xb4\x08\xb3\xb3\xb3\x08\xb7\xb7\xb7\t\xa1\xa1\xa1\x07\xe9\xe8\xeb\x07\xbc\xbc\xbc\t\xb3\xb3\xb3\x08\xb4\xb4\xb4\x08\xbd\xbd\xbd\t\x8d\x8d\x8d\x08\xda\xda\xda\x0c\xf6\xf6\xf8\r\x8d\x8d\x8d\x08\xbd\xbd\xbd\t\xb4\xb4\xb4\x08\xb4\xb4\xb4\x08\xb4\xb4\xb4\x08\xb4\xb4\xb4\x08\xb4\xb4\xb4\x08\xbc\xbc\xbc\t\xf8\xf8\xf8\t\xe6\xe6\xe6\x0c\xfc\xfc\xfd=\xff\xff\xff\xb9\xe8\xe7\xee\xfe\x12\x12\x17\xfc\x00\x00\x00\xffXW`\xffKJO\xff\x00\x00\x00\xfeUUV\xff\xff\xff\xff\xf0\xcc\xcd\xcc\x16\x00\x00\x00\x00\xff\xff\xff\x01\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa666\xff\x00\x00\x00\xffhhi\xff+++\xff\x8f\x8f\x8f\xfe\xfe\xfe\xfe\xff\xff\xff\xff\xf1\xff\xff\xff\xe4\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe4\xff\xff\xff\xe4\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe5\xff\xff\xff\xe9\xff\xff\xff\xeb\xff\xff\xff\xe5\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xff\xff\xff\xe6\xfb\xfb\xfb\xe8\xff\xff\xff\xff\xd3\xd3\xd3\xff@@B\xfe\x00\x00\x00\xff\x02\x02\x01\xfe\x84\x85\x84\xff\x12\x12\x11\xfe\x00\x00\x00\xfd\x90\x90\x91\xfe\xff\xff\xff\xda\xb9\xb8\xb9\x06\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xfb\xfb\xfb%\xff\xff\xff\xfa888\xff\x00\x00\x00\xffVVW\xff\xc0\xc0\xc0\xff\xff\xff\xff\xff\xff\xff\xff\xff\xc6\xc6\xc6\xffccc\xffddd\xffddd\xffddd\xff```\xff\x82\x81\x83\xff\xc1\xc0\xc3\xff[[Z\xffdde\xffccc\xffeee\xff___\xff\x93\x93\x93\xff\xb2\xb2\xb6\xffVVV\xfffff\xffddd\xffddd\xffddd\xffddd\xffddd\xffddd\xff```\xff]]]\xff>>>\xfd\x08\x08\x08\xfc\x00\x00\x00\xff\x00\x00\x00\xfewu\x87\xffKJO\xfe\x00\x00\x00\xff\x04\x04\x05\xfa\xdf\xdf\xe0\xff\xff\xff\xff\x8d\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\xfa\xfa\xfa$\xff\xff\xff\xf9333\xff\x00\x00\x00\xfe\xa9\xa9\xa9\xff\xff\xff\xff\xff\xfa\xfa\xfa\xfe\xb9\xb9\xb9\xff\x04\x04\x04\xfe\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfdMLS\xfdbbe\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfdfdk\xfdKJP\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfd\x00\x00\x00\xfe\x00\x00\x00\xfe\'\'\'\xfe\x7f~\x81\xffYWb\xfe\x00\x00\x00\xff\x00\x00\x00\xfd\x84\x83\x8a\xfa\xff\xff\xff\xf6\xe7\xe7\xe7\'\x00\x00\x00\x00\xff\xff\xff\x01\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\xfc\xfc\xfc%\xfc\xfc\xfc\xf8***\xfe\x8e\x8e\x8f\xff\xff\xff\xff\xfe\xff\xff\xff\xff\xe0\xe0\xe0\xff666\xff\'&\'\xff++,\xff,,-\xff""#\xff\x00\x00\x00\xff\x05\x04\x05\xff\x04\x04\x04\xff\x00\x00\x00\xff__d\xffvux\xff\x00\x00\x00\xff\x05\x05\x05\xff\x03\x03\x03\xff\x03\x03\x03\xff\x00\x00\x00\xffxw|\xffffj\xff***\xff,,-\xff++,\xff++,\xff++,\xff++,\xff,+,\xff777\xffUUY\xffyx\x81\xff}|}\xff222\xfe\x00\x00\x00\xff\x00\x00\x00\xfeZZ[\xf8\xfb\xfb\xfb\xff\xf6\xf5\xf6w\xfa\xfa\xfa\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\xff\xff\xff$\xf7\xf7\xf7\xff\xc2\xc2\xc2\xff\xff\xff\xff\xfe\xff\xff\xff\xff\xcc\xcc\xcc\xffmmn\xffmmn\xffrrs\xffppr\xffmmm\xff\x8a\x89\x94\xff\\[m\xff\x00\x00\x00\xff\x06\x06\x05\xff\x08\x08\x08\xff\x00\x00\x00\xffa`e\xffxxz\xff\x00\x01\x00\xff\x04\x04\x04\xff\x05\x05\x05\xff\x05\x05\x06\xff\x00\x00\x00\xff^]a\xffsst\xffoop\xffppq\xffppq\xffppq\xffppq\xffnnp\xffhhh\xffNMT\xfe\x1f\x1e\x1f\xfe\x00\x00\x00\xfe\x00\x00\x00\xfe\x0e\x0e\x0e\xfa\x81\x80\x87\xfd\xfd\xfd\xfd\xff\xfb\xfb\xfb\x96\xfa\xfa\xfa\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x01\x00\x00\x00\x00\xff\xff\xff\t\xff\xff\xff\xb6\xff\xff\xff\xf1\xfb\xfb\xfb\xf9\xaf\xae\xaf\xfa\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb`^v\xfbKJ^\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfbHGL\xfb__b\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb002\xfd{{|\xff\xd6\xd6\xd7\xff\xff\xff\xff\xf7\xfc\xfc\xfcy\xfd\xfd\xfe\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\xfb\xfb\xfbj\xfc\xfc\xfc\xff\xa8\xa7\xa8\xff\x97\x97\x97\xff\x9f\x9f\x9f\xff\x9d\x9d\x9d\xff\x9d\x9d\x9d\xff\x9d\x9d\x9d\xff\x9e\x9e\x9e\xff\x9b\x9b\x9b\xff\x95\x95\x95\xff\xde\xdd\xeb\xff\xa4\xa4\xa7\xff\x99\x99\x99\xff\x9a\x9a\x9a\xff\x9d\x9d\x9d\xff\x90\x90\x90\xff\xd7\xd7\xd9\xff\xaf\xaf\xaf\xff\x98\x98\x98\xff\x9b\x9b\x9b\xff\x9a\x9a\x9a\xff\x9d\x9d\x9d\xff\x9d\x9d\x9d\xff\x9d\x9d\x9d\xff\x9d\x9d\x9d\xff\x9d\x9d\x9d\xff\x9d\x9d\x9d\xff\x9d\x9d\x9d\xff\xa0\xa0\xa1\xff\xa9\xa9\xa9\xff\xc0\xc0\xc1\xff\xdd\xdd\xde\xff\xfd\xfd\xfe\xff\xff\xff\xff\xe9\xff\xff\xff\x93\xfa\xfa\xfa.\xfe\xfe\xfe\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x01\xff\xff\xff\x01\xff\xff\xff\r\xff\xff\xffq\xff\xff\xff\x98\xff\xff\xff\x9b\xff\xff\xff\x9b\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9b\xff\xff\xff\x9c\xff\xff\xff\x8c\xff\xff\xff\x97\xff\xff\xff\x9c\xff\xff\xff\x9b\xff\xff\xff\x9b\xff\xff\xff\x9c\xff\xff\xff\x8e\xff\xff\xff\x93\xff\xff\xff\x9c\xff\xff\xff\x9b\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9c\xff\xff\xff\x9b\xff\xff\xff\x99\xff\xff\xff\x94\xff\xff\xff\x87\xff\xff\xff~\xfe\xfe\xfeH\xf5\xf5\xf5\x14\xf9\xf9\xf9\x00\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x03\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xf0\x00\x00\x00?\xff\x00\x00\xe0\x00\x00\x00\x01\xff\x00\x00\xe0\x00\x00\x00\x00\x7f\x00\x00\xe0\x00\x00\x00\x00\x1f\x00\x00\xe0\x00\x00\x00\x00\x0f\x00\x00\xe0\x00\x00\x00\x00\x0f\x00\x00\xe0\x00\x00\x00\x00\x07\x00\x00\xe0\x00\x00\x04\x00\x07\x00\x00\xe0\x1f\xff\xff\xe0\x07\x00\x00\xe0\x1f\xff\xff\xf0\x07\x00\x00\xe0\x1f\xff\xff\xf8\x07\x00\x00\xe0\x1f\xff\xff\xf8\x07\x00\x00\xe0\x1f\xff\xff\xf0\x07\x00\x00\xe0\x1f\xfe\x00\x00\x07\x00\x00\xe0\x1f\xfc\x00\x00\x07\x00\x00\xe0\x1f\xf8\x00\x00\x0f\x00\x00\xe0\x1f\xf0\x00\x00\x0f\x00\x00\xe0\x1f\xe0\x00\x00\x0f\x00\x00\xe0\x1f\xe0\x00\x00\x0f\x00\x00\xe0\x1f\xe0\x00\x00\x07\x00\x00\xe0\x1f\xe0\x00\x00\x07\x00\x00\xe0\x1f\xff\xff\xe0\x07\x00\x00\xe0\x1f\xff\xff\xf8\x07\x00\x00\xe0\x1f\xff\xff\xf8\x07\x00\x00\xe0\x1f\xff\xff\xf8\x07\x00\x00\xe0\x1f\xff\xff\xf0\x07\x00\x00\xe0\x00\x00\x00\x00\x07\x00\x00\xe0\x00\x00\x00\x00\x07\x00\x00\xe0\x00\x00\x00\x00\x0f\x00\x00\xe0\x00\x00\x00\x00\x0f\x00\x00\xe0\x00\x00\x00\x00\x1f\x00\x00\xe0\x00\x00\x00\x00?\x00\x00\xf0\x00\x00\x00\x00\xff\x00\x00\xf8\x00\x00\x00\x07\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00(\x00\x00\x00 \x00\x00\x00@\x00\x00\x00\x01\x00 \x00\x00\x00\x00\x00\x80\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00uuw\x00\x00\x00\x00\x00\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x03\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x03\xff\xff\xff\x03\xff\xff\xff\x04\xff\xff\xff\x04\xff\xff\xff\x04\x00\x00\x00\x00\xc9\xc9\xc8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x02\xff\xff\xff\x01\xfa\xfa\xfa\xa9\xe2\xe2\xe2\xde\xe2\xe2\xe3\xdc\xe2\xe2\xe2\xde\xe2\xe2\xe2\xde\xe2\xe2\xe2\xde\xe2\xe2\xe2\xde\xe2\xe2\xe2\xde\xe2\xe2\xe2\xde\xe2\xe2\xe2\xde\xd8\xd8\xd8\xde\xf4\xf4\xf4\xdc\xe3\xe3\xe3\xde\xe2\xe2\xe3\xde\xdb\xdb\xdb\xde\xea\xea\xea\xdd\xec\xec\xec\xdd\xe0\xe0\xe0\xde\xe2\xe1\xe2\xdb\xda\xda\xda\xde\xfc\xfc\xfc\x8a\xff\xff\xffd\xff\xff\xff\x93\xff\xff\xffY\xff\xff\xff\x15\xf9\xf9\xf9\x00\xff\xff\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x06\xde\xde\xde\xec!!!\xff\r\r\r\xff\x0f\x0f\x0f\xff\x10\x10\x10\xff\x10\x10\x10\xff\x10\x10\x10\xff\x10\x10\x10\xff\x11\x11\x11\xff\x11\x11\x11\xffIIJ\xff^^^\xff\x16\x16\x17\xff\x10\x10\x10\xff##%\xffqqv\xff$$$\xff\x11\x11\x12\xff\x08\x08\t\xff\x96\x96\x96\xff\xff\xff\xff\xff\xa8\xa8\xa8\xffOOR\xff\x9a\x9a\x9b\xff\xed\xed\xef\xe9\xff\xff\xffq\xf8\xf8\xf8\x00\xff\xff\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x06\xdd\xdd\xdd\xe0\x00\x00\x00\xfe777\xfdWVW\xfeNNO\xfeQQR\xfeQQR\xfeQQR\xfePPP\xfeXXX\xfeKKK\xfe\x00\x00\x00\xfe\x00\x00\x00\xfe\x10\x10\x12\xfecbi\xfe\x14\x13\x15\xfe\x00\x00\x00\xfe\x00\x00\x00\xfc\x91\x91\x91\xfe\xff\xff\xff\xfe\xd0\xd0\xd0\xfe[[]\xfd333\xfb\x04\x04\x04\xfb\x16\x16\x18\xff\xa9\xa9\xa9\xff\xff\xff\xfe\x96\xf6\xf6\xf6\x00\xfd\xfd\xfd\x01\xfe\xfe\xfe\x00\x00\x00\x00\x00\xff\xff\xff\x05\xde\xde\xde\xe1\x05\x05\x05\xffQQQ\xfe\x18\x18\x18\xfe\x00\x00\x00\xff\x00\x00\x00\xfe\x00\x00\x00\xfe\x00\x00\x00\xfe\x00\x00\x00\xfe\x00\x00\x00\xfe\x00\x00\x00\xfe\x00\x00\x00\xfe\x08\x08\n\xfeSSY\xfe\x06\x06\x07\xfe\x00\x00\x00\xfe\x05\x04\x05\xff\x9b\x9a\x9b\xff\xff\xff\xff\xff\x9e\x9e\x9f\xfe\x00\x00\x00\xfe\x00\x00\x00\xfb+++\xfebae\xff,+2\xfd\x05\x05\x05\xf8\xc4\xc4\xc7\xff\xfb\xfb\xfbG\x00\x00\x00\x00\xfe\xfe\xff\x06\x00\x00\x00\x00\xff\xff\xff\x06\xde\xde\xde\xe2\x04\x04\x04\xffTTT\xfe\x03\x03\x03\xffooo\xff\xab\xab\xab\xff\xa4\xa4\xa4\xff\xa7\xa7\xa7\xff\xab\xab\xab\xff\xab\xab\xab\xff\xaf\xaf\xaf\xff\xaa\xaa\xaa\xff\xcb\xca\xcd\xff\xbe\xbe\xbf\xff\xa7\xa7\xa7\xff\xaa\xaa\xab\xff\xc5\xc5\xc5\xff\xff\xff\xff\x96\xf6\xf6\xf7\xd2\xb3\xb3\xb3\xff\xa8\xa8\xa8\xffwww\xff\n\n\n\xff\x00\x00\x00\xfedcl\xff\t\t\t\xfbJIK\xfd\xff\xff\xff\xb7\xda\xda\xda\x00\xfe\xfe\xfd\x07\x00\x00\x00\x00\xff\xff\xff\x06\xdd\xdd\xdd\xe2\x07\x07\x07\xff\'\'\'\xfeNNO\xfe\xc0\xc0\xc0\xff\xff\xff\xffW\xff\xff\xff3\xff\xff\xff5\xff\xff\xff3\xff\xff\xff3\xff\xff\xff3\xff\xff\xff2\xff\xff\xff8\xff\xff\xff7\xff\xff\xff2\xff\xff\xff3\xff\xff\xff0\xf6\xf6\xf6\x00\xff\xff\xff#\xff\xff\xff7\xff\xff\xff6\xff\xff\xff\x87\xcc\xcc\xcd\xfb\n\n\t\xfd  $\xfe??F\xfd\x0c\x0c\x0b\xfe\xf0\xf0\xf1\xdbVVV\x02\xdb\xda\xe3\x01\x00\x00\x00\x00\xff\xff\xff\x06\xdb\xdb\xdb\xe2\x10\x10\x10\xff\x00\x00\x00\xfe!!"\xfe\xd4\xd4\xd5\xfe\xfb\xfb\xfb"\xfb\xfb\xfb\x00\xfb\xfb\xfb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\xfb\xfb\xfb\x00\xfa\xfa\xfa\x00\xff\xff\xff\x97ccc\xff\x00\x00\x00\xfaONO\xfe\x06\x06\x06\xff\xe4\xe4\xe4\xec\xff\xff\xff\x0f\xfe\xfe\xff\x00\x00\x00\x00\x00\xfc\xfc\xfc\x05\xde\xde\xde\xe1\x08\x08\x08\xff\x01\x01\x01\xfd\x00\x00\x00\xfe\xaf\xb0\xaf\xfe\xff\xff\xff.\xfe\xfe\xfe\x02\xff\xff\xff\x04\xff\xff\xff\x02\xff\xff\xff\x02\x00\x00\x00\x02\xff\xff\xff\x02\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfb\xfb\xfb\x00\xfa\xfa\xfa\x00\xff\xff\xff\x92ggf\xfe\x00\x00\x01\xfaMMP\xfd\r\r\r\xff\xea\xea\xeb\xdciii\x03\xcb\xcb\xd0\x01\x00\x00\x00\x00\xdb\xdb\xdb\x05\xec\xec\xec\xe0[[[\xff\x00\x00\x00\xfd\x01\x01\x01\xfd\xb4\xb4\xb4\xff\xff\xff\xff-\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x01\xfd\xfd\xfd\x00\xff\xff\xfff\xff\xff\xff\x91\xff\xff\xff\x8f\xff\xff\xff\x91\xff\xff\xff\x91\xff\xff\xff\x91\xff\xff\xff\x92\xff\xff\xff\x90\xff\xff\xff\xaa\xcd\xcd\xce\xfc\x03\x03\x02\xfe99:\xfe,,-\xfc"""\xfe\xfb\xfb\xfb\xccvuv\x00\xf2\xf2\xf3\x04\x00\x00\x00\x00\xff\xff\xff\x06\xd2\xd2\xd2\xe2@@@\xffWWX\xfe\x00\x00\x00\xfe\xb1\xb1\xb1\xff\xff\xff\xff,\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf9\xf9\xf9\x00\xff\xff\xff\x86\xb2\xb2\xb2\xffggg\xffjjj\xffhhh\xffmmm\xffuuu\xffqqq\xffddd\xffAAC\xff\x02\x02\x03\xff,+1\xffcbl\xff\x00\x00\x00\xfallm\xff\xff\xff\xff\x9b\x00\x00\x00\x00\xff\xff\xff\n\x00\x00\x00\x00\xff\xff\xff\x05\xdd\xdd\xdd\xe1\x00\x00\x00\xff555\xfdIII\xfd\xb1\xb1\xb1\xff\xff\xff\xff-\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\xff\xff\xff\x01\xfe\xfe\xfe\x03\xff\xff\xff\x8c\xdb\xdb\xdc\xff===\xfa!!!\xfa   \xfb#"#\xfb\x0c\x0c\x0c\xfb\x00\x00\x00\xfb\x00\x00\x00\xfb\x08\x08\t\xfb000\xfbXX[\xfeIHO\xfe\x05\x05\x07\xfe///\xfd\xe1\xe1\xe1\xf1\xf2\xf2\xf2\'\xfb\xfb\xfb\x00\xfe\xfe\xfe\x03\x00\x00\x00\x00\xff\xff\xff\x06\xdb\xdb\xdb\xe2\x11\x11\x11\xff\x00\x00\x00\xfe(()\xfe\xd8\xd8\xd8\xff\xff\xff\xff,\x00\x00\x00\x00\xff\xff\xff\x01\xff\xff\xff\x03\x00\x00\x00\x00\xfe\xfe\xfeK\xba\xba\xba\xff\x1d\x1d\x1d\xf5\x1c\x1c\x1c\xfa&%&\xfb&&&\xfb\'\'\'\xfbMMM\xfb)))\xfb\x00\x00\x00\xfb(()\xfb`_`\xfdppq\xffDDD\xfe\x06\x06\x06\xfe444\xfd\xe8\xe8\xe9\xee\xef\xef\xef!\xfb\xfb\xfb\x00\xfe\xfe\xfe\x02\x00\x00\x00\x00\xfb\xfb\xfb\x05\xdf\xdf\xdf\xe1\t\n\t\xff\x00\x00\x00\xfd\x00\x00\x00\xfe\xb1\xb1\xb1\xff\xff\xff\xff,\x00\x00\x00\x00\xff\xff\xff\x01\xff\xff\xff\x04\x00\x00\x00\x00\xff\xff\xff`\xa9\xa9\xa9\xffBBB\xffHIH\xffEEE\xffFFF\xffFFF\xffLLL\xff\x97\x97\x97\xff\\\\\\\xff===\xff\x1a\x1a\x1a\xff\x00\x00\x00\xfe../\xffbaf\xff\x00\x00\x00\xfauuu\xff\xff\xff\xff\x93\x00\x00\x00\x00\xff\xff\xff\n\x00\x00\x00\x00\xe3\xe4\xe0\x05\xeb\xeb\xeb\xe1__b\xff\x02\x02\x03\xfd\x01\x01\x01\xfd\xb3\xb3\xb3\xff\xff\xff\xff-\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x1b\xff\xff\xff\x8c\xff\xff\xff\x93\xff\xff\xff\x93\xff\xff\xff\x94\xff\xff\xff\x94\xff\xff\xff\x94\xff\xff\xff\x94\xff\xff\xff\x93\xff\xff\xff\x92\xff\xff\xff\x9c\xfc\xfc\xfc\xd7\x94\x94\x94\xff\x00\x00\x00\xfe??C\xff((*\xfc(((\xfe\xfd\xfd\xfd\xc8\x97\x97\x9a\x00\xf8\xf8\xf9\x05\x00\x00\x00\x00\xff\xff\xff\x06\xd2\xd2\xd2\xe2::;\xffSSU\xfe\x00\x00\x00\xfe\xb3\xb3\xb3\xfe\xff\xff\xff-\x00\x00\x00\x00\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe1\xe1\xe1\x00\xfc\xfc\xfc\x00\xfd\xfd\xfd\x07\xff\xff\xff\xbaIIJ\xfe\x06\x06\x05\xfbHHH\xfd\x0e\x0e\x11\xff\xed\xed\xf0\xd8??O\x01\xcb\xca\xde\x01\x00\x00\x00\x00\xff\xff\xff\x05\xde\xde\xde\xe1\x00\x00\x00\xffUTV\xfd\x02\x02\x02\xfd\xac\xac\xac\xff\xfd\xfd\xfd(\xfc\xfc\xfc\x00\xfd\xfd\xfd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\xfd\xfd\x02\xfc\xfc\xfc\x00\xfa\xfa\xfatyyy\xff\x00\x00\x00\xf9OOP\xfe\x06\x06\x06\xff\xe4\xe4\xe3\xec\xff\xff\xff\x0e\xff\xff\xfe\x00\x00\x00\x00\x00\xff\xff\xff\x06\xdd\xdd\xdd\xe2\x07\x07\x07\xffKKL\xfe\x00\x00\x00\xfe\xba\xba\xba\xfe\xff\xff\xff=\xff\xff\xff\x16\xff\xff\xff\x18\xff\xff\xff\x16\xff\xff\xff\x14\xff\xff\xff\x16\xff\xff\xff\x16\xff\xff\xff\x15\xff\xff\xff\x18\xff\xff\xff\x18\xff\xff\xff\x15\xff\xff\xff\x16\xff\xff\xff\x16\xff\xff\xff\x16\xff\xff\xff\x16\xff\xff\xff\x17\xff\xff\xff5\xff\xff\xff\xcd==@\xfe\x0c\x0c\r\xfcFFK\xfd\n\n\n\xfe\xef\xef\xef\xdehhi\x04\xe6\xe5\xe7\x01\x00\x00\x00\x00\xff\xff\xff\x05\xde\xde\xde\xe1\x07\x07\x07\xff>>>\xfe\x96\x96\x96\xff\xff\xff\xff\xfe\xe0\xe0\xe0\xf4\xcf\xcf\xcf\xf3\xd2\xd2\xd2\xf3\xcf\xcf\xd0\xf3\xea\xe9\xeb\xf1\xc8\xc8\xc7\xf3\xd0\xd0\xd0\xf3\xcc\xcc\xcc\xf2\xdf\xdf\xe0\xf4\xdc\xdc\xdc\xf5\xc8\xc8\xc7\xf2\xd2\xd2\xd2\xf3\xd1\xd1\xd1\xf3\xd1\xd1\xd1\xf3\xd4\xd4\xd4\xf3\xcf\xcf\xcf\xf2\xba\xba\xba\xff\\[\\\xff\x00\x00\x00\xfe]]b\xff\x0f\x0f\x0f\xfbHHI\xfd\xff\xff\xff\xb7\xd8\xd8\xd9\x00\xfc\xfc\xfc\x07\x00\x00\x00\x00\xff\xff\xff\x05\xdc\xdc\xdc\xdf\x00\x00\x00\xfe\x99\x99\x99\xfd\xff\xff\xff\xfe\xa8\xa8\xa8\xff\x05\x05\x05\xff\x00\x00\x00\xff\x00\x00\x00\xff\x05\x05\x05\xffBBE\xffHHJ\xff\x00\x00\x00\xff\x04\x04\x04\xff\x13\x13\x13\xffa`e\xff\x17\x17\x18\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xff\x00\x00\x00\xfe\x0c\x0c\r\xfd\\\\_\xff98A\xfe\x00\x00\x00\xf8\xc1\xc1\xc4\xff\xfc\xfc\xfcK\x00\x00\x00\x00\xff\xff\xff\x06\x00\x00\x00\x00\xf8\xf8\xf8\x06\xd3\xd3\xd3\xed\x92\x92\x93\xff\xff\xff\xff\xfd\xd0\xcf\xd0\xffLLM\xfeGGH\xfeQQR\xff32:\xfe\x00\x00\x00\xfe\x00\x00\x00\xfeFEH\xfeHHJ\xfe\x00\x00\x00\xfe\x00\x00\x00\xfe\x13\x13\x14\xff`_c\xffONP\xffKKL\xffLKL\xfeKKL\xffLLL\xfeVVZ\xfeKKN\xfd\x12\x12\x11\xfb\x05\x05\x05\xfd\x9b\x9b\x9c\xff\xff\xff\xff\x9c\xf6\xf6\xf6\x00\xfd\xfd\xfd\x01\xfd\xfd\xfd\x00\xff\xff\xff\x02\xe9\xe9\xe9\x00\xfa\xfa\xfa\xa4\xff\xff\xff\xff\xaf\xae\xaf\xff\x0f\x0f\x0f\xff\x15\x14\x15\xff\x17\x17\x18\xff\x18\x18\x18\xffRP`\xff@?L\xff\x03\x03\x03\xff\x00\x00\x00\xffGFJ\xffEEG\xff\x04\x04\x04\xff\x00\x00\x00\xff\x04\x04\x04\xff\x17\x17\x17\xff\x15\x15\x16\xff\x16\x16\x16\xff\x16\x16\x16\xff\x16\x16\x17\xff\x16\x16\x17\xff&&&\xffccc\xff\xd3\xd3\xd4\xfe\xff\xff\xff\x8e\xf7\xf7\xf7\x00\xff\xff\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfe\xfe\xfe\x00\xfc\xfc\xfc\x94\xcc\xcc\xcc\xe5\xc3\xc3\xc3\xe5\xc4\xc4\xc4\xe8\xc3\xc3\xc3\xe8\xc2\xc2\xc2\xe8\xc2\xc2\xc1\xe8\xe7\xe6\xed\xe2\xc9\xc9\xc9\xe8\xc9\xc9\xc9\xe8\xc0\xc0\xc0\xe8\xe6\xe6\xe7\xe2\xca\xca\xca\xe7\xc9\xc9\xc9\xe8\xc7\xc7\xc7\xe8\xc2\xc2\xc2\xe8\xc3\xc3\xc3\xe8\xc3\xc3\xc3\xe8\xc3\xc3\xc2\xe8\xc5\xc5\xc5\xe6\xd8\xd8\xd9\xdf\xfd\xfd\xfe\xce\xff\xff\xff\x8e\xff\xff\xff2\xfa\xfa\xfa\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe2\xe2\xe2\x00\xff\xff\xff\x01\xfe\xfe\xfe\x00\xff\xff\xff\t\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x07\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x07\xff\xff\xff\n\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\x0b\xff\xff\xff\n\xff\xff\xff\x05\xa4\xa1\xab\x00\x00\x00\x00\x00\xcb\xcb\xcb\x00\xfe\xfe\xfe\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfa\xfa\xfb\x00\xff\xff\xff\x03\xff\xff\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xc0\x00\x00\x7f\xc0\x00\x00\x0f\xc0\x00\x00\x07\xc0\x00\x00\x07\xc0\x00\x00\x03\xc1\xff\xfe\x03\xc1\xff\xff\x03\xc1\xff\xff\x03\xc1\xfe\x00\x03\xc1\xfc\x00\x03\xc1\xf8\x00\x07\xc1\xf8\x00\x07\xc1\xf0\x00\x03\xc1\xf8\x00\x03\xc1\xff\xff\x03\xc1\xff\xff\x03\xc1\xff\xff\x03\xc0\x00\x00\x03\xc0\x00\x00\x07\xc0\x00\x00\x07\xc0\x00\x00\x0f\xe0\x00\x00?\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff(\x00\x00\x00\x10\x00\x00\x00 \x00\x00\x00\x01\x00 \x00\x00\x00\x00\x00@\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xac\xac\xac\x00\x00\x00\x00\x00\xb6\xb6\xb6\x00\xb6\xb6\xb6\x00\xb6\xb6\xb6\x00\xb5\xb5\xb5\x00\xbf\xbf\xbf\x00\xb4\xb4\xb5\x00\xbb\xbb\xbc\x00\xb5\xb5\xb6\x00\xb4\xb4\xb5\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfc\xfc\xfc\x00\xee\xee\xee-\xf7\xf7\xf8H\xf4\xf4\xf4H\xf5\xf5\xf5H\xf3\xf3\xf3H\xff\xff\xffG\xff\xff\xffH\xff\xff\xffH\xff\xff\xffH\xdd\xdd\xddE\xff\xff\xff!\xff\xff\xff\x16\x00\x00\x00\x00\x00\x00\x00\x00\xf5\xf5\xf5\x00000\x00\x9c\x9c\x9c\xc0FEF\xffJJJ\xffEEF\xffHHH\xffOOP\xff//0\xffPPR\xff889\xff\xac\xac\xac\xff\xb5\xb5\xb5\xf8uuv\xf1\xda\xda\xda\x95\xdc\xdc\xdc\x05\x00\x00\x00\x00\x00\x00\x00\x00\x86\x86\x86\xba$$$\xff999\xffMMN\xffKKK\xff===\xffYY[\xff@@@\xff\x7f\x7f\x80\xf0\xd0\xd0\xd0\xfcQQQ\xff\x1d\x1c\x1f\xff;;>\xff\xcd\xcd\xcdy\xa4\xa4\xa5\x00\x00\x00\x00\x00\x83\x83\x83\xb9\x16\x16\x16\xff\xb2\xb2\xb3\xc6UUV5\xff\xff\xff6\xff\xff\xff6\xff\xff\xff1\xff\xff\xff\'\xff\xff\xff\x10\xfe\xfe\xff\x1d\xdc\xdc\xdcIZZZ\xf1\x1f\x1e!\xff\x95\x95\x95\xbf\x00\x00\x00\x00\t\t\t\x00\x9b\x9b\x9b\xb8\x06\x06\x06\xff\x90\x90\x90\xb3\x00\x00\x00\x00\xcb\xcb\xcc\x00\xe9\xe9\xe9\x00\xff\xff\xff\x05\xff\xff\xff2\xff\xff\xff6\xff\xff\xff3\xdc\xdc\xdc?jij\xec\x1f\x1f \xff\x97\x97\x97\xb9\x05\x05\x05\x00\x07\x07\x07\x00\x92\x92\x92\xb8,,,\xff\x9b\x9b\x9b\xb7\x10\x10\x10\x00\xf3\xf3\xf3\x00\xd5\xd5\xd5%\xa8\xa8\xa9\xdcPPP\xffMMM\xff222\xffOOP\xff//3\xff889\xff\xd4\xd4\xd3q\xad\xad\xae\x00\x00\x00\x01\x00\x8a\x8a\x8a\xb8\x06\x06\x06\xff\xa1\xa1\xa1\xb6\x11\x11\x11\x00\x8e\x8e\x8e\x00\xcd\xcd\xce\xa2WWW\xffAAB\xffKKK\xff666\xffDDE\xff,,-\xff;;<\xff\xd7\xd7\xd7m\xb5\xb5\xb5\x00\r\r\x0f\x00\xa2\xa2\xa3\xb8\x1c\x1c\x1d\xff\x87\x87\x87\xb3\x00\x00\x00\x00\xde\xde\xde\x00\xf3\xf3\xf3\x19\xe5\xe5\xe45\xf9\xf9\xf95\xfc\xfc\xfb5\xff\xff\xff5\xd8\xd8\xd8UYYY\xf1  !\xff\x98\x98\x99\xb7\n\n\x0c\x00\x00\x00\x00\x00\x86\x86\x86\xb9###\xff\xab\xab\xab\xc1111&\xff\xff\xff&\xff\xff\xff\x1e\xff\xff\xff\x19\xff\xff\xff\x19\xff\xff\xff\x18\xff\xff\xff\x18\xda\xda\xda\'tsu\xe8\x1d\x1d\x1f\xff\x95\x95\x95\xc0\x00\x00\x00\x00\x01\x01\x01\x00~~~\xc1\x94\x94\x94\xff\xae\xae\xae\xfeNNN\xfeUUV\xfeaab\xfeLLL\xffjjl\xffRQR\xfePPP\xfeRRR\xff\'\'*\xff77:\xff\xcd\xcd\xcdz\xa3\xa3\xa4\x00\xd3\xd3\xd3\x00\xe6\xe6\xe6\x94\xc3\xc3\xc3\xffOOO\xffJJN\xff337\xff??@\xff222\xff334\xffCCD\xff>>>\xffFFG\xfffff\xfd\xd3\xd3\xd3\xa2\xd5\xd5\xd5\x06\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\x03\xd3\xd3\xd3J\xd1\xd1\xd1N\xe3\xe3\xe4N\xf9\xf8\xfcM\xe2\xe2\xe2N\xfb\xfb\xfbL\xdf\xdf\xdfN\xdd\xdd\xdcN\xdd\xdd\xddN\xea\xea\xeaJ\xff\xff\xff)\x00\x00\x00\x00\x00\x00\x00\x00\xf3\xf3\xf4\x00\xfc\xfc\xfc\x00\x00\x00\x00\x00{{{\x00\x9f\x9f\x9f\x00\x9f\x9f\xa0\x00\xac\xac\xb0\x00\x9d\x9d\x9d\x00\xad\xad\xad\x00\xa0\xa0\xa0\x00\xa0\xa0\xa0\x00\x99\x99\x99\x00\x97\x97\x97\x00\xcc\xcb\xcd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\x00\x00\xff\xff\x00\x00\xff\xff\x00\x00\x80\x03\x00\x00\x80\x01\x00\x00\x8f\xf1\x00\x00\x8f\xf1\x00\x00\x8e\x01\x00\x00\x8c\x01\x00\x00\x8f\xf1\x00\x00\x8f\xf1\x00\x00\x80\x01\x00\x00\x80\x03\x00\x00\xff\xff\x00\x00\xff\xff\x00\x00\xff\xff\x00\x00')

15086

In [59]:
data = unhex('c520bca5cd3e3e89a0cc855b6c07abae452b39828382ad906bc8a97fbc9c4991143e3cfaf58e5b3529ca694ae4c909589312983f90b2a00d90b1701f50b5481fe061e6012f2ad0701fb7457dbaf35ada5dc96770e5c6a318d8fc01dbcfbf9c23d5ac331a')

decrypt(data, seed)




b'Q\xc2\xee\xcc\x9f\xb7twh\xe7\x82JF]\xbeU\xa9\x9a\x94.\xbeM\n\xc4+/\x17\xec\xad\x11>e\x10\x82u\xb0\xa3\xac\xffeQ,\x807\xa9O\x87\x04O\x1a1\x96b.\xfa\xbc\xe0Zgh\xde\xdf\xf1\xb9\x94\xd3 \xb1O\x18\xfd^7\xf5\xc3&*\xb1A\xe6\x91\xa9\xe3\x986\xd3\x0e?x\xee\xa2\x1e\xae\xa1\x19\xf61\xab\t\xb3'

#### Shellcode 
Some of the encrypted data is shellcode that is used for thread injection, and token stealing. This code is not compressed. The shellcode files are listed below and can be downloaded from Malshare.

```
0d2104672c11ff0822bf096e97225ceab74f1e2ecff7a2a6fde152a081b90b87.bin
2df9218c31586e1ce85d989ea97d9e6a6664d4a50b681914a5b6c44064d5c146.bin
d8dd2530a4b2165f16547c237e43dc15adb6e168af5ae8b153793d7aee863d93.bin
da2d72e132f9e8706676a72d9bf48a2bd26d9f8014686424ddb927b87637f2a1.bin
```

#### Icon
One of the blobs contains an aplib compressed .ico file that is used as a custom icon by Lockbit 3.0.

![](https://i.imgur.com/9i2aYW7.png)

#### PE Files
The blobs also contain 3 PE files that are compressed with aplib, the files are listed below (and can be downloaded from Malshare).

```
63c8efca0f52ebea1b3b2305e17580402f797a90611b3507fab6fffa7f700383_pe.bin
917e115cc403e29b4388e0d175cbfac3e7e40ca1742299fbdb353847db2de7c2_pe.bin
d641ad955ef4cff5f0239072b3990d47e17b9840e07fd5feea93c372147313c5_pe.bin
```

### TODO: 
- print RSA key, and affiliate in a nice format
- print key matrix data
- analyze extracted PE files

## Analysis of The Embedded PE Files




Our samples hash: `d641ad955ef4cff5f0239072b3990d47e17b9840e07fd5feea93c372147313c5`.

The sample uses a dynamic API resolving routine similar to the main Lockbit binary, all APIs are resolved in a "API table", though the actual API hash algorithm is different.


[@c3rb3ru5d3d53c](https://gist.github.com/c3rb3ru5d3d53c) converted this into a [python script](https://gist.github.com/c3rb3ru5d3d53c/4f2c984d81ef64e5f133e37726619c64) that will generate the API hashes for you. She also has a nice [blog post](https://c3rb3ru5d3d53c.github.io/malware-blog/lockbit-v3-api-hashing/).

### The API Hash Algorithm

The hash algorithm has a hard coded key which can be modified via an optional seed, the rest of the algorthm is a simple ROT-13 add, same as the lockbit main. This hash algorithm is used to hash the lowercase string of the DLL, which is then used as the seed for the API hashes. 

Example from c3rb3ru5d3d53c's blog post with xor key 0x3B029FD4:
```
0xaae0cefb -> kernel32.dll FindFirstFileExW
```

```python
def ror(n,rotations=1,width=32):
    return (2**width-1)&(n>>rotations|n<<(width-rotations))


def hash_algo(string, seed):
    string = string + b'\x00'
    result = 0xC8B32494 ^ seed
    for c in string:
        result = ror(result, rotations=0x0d, width=32)
        result = (result + c) & 0xffffffff
        if c == 0x00: break
    return result


def get_api_hash(dll_name, api_name, seed, xor_key):
    dll_name = dll_name.lower()
    dll_hash = hash_algo(dll_name, seed)
    tmp_hash = hash_algo(api_name, dll_hash)
    return (~tmp_hash & 0xffffffff) ^ xor_key
```

This algorithm is now in [hashdb](https://github.com/OALabs/hashdb).


#### Obfuscated VTables

For some reason the developers create "obfuscated" jumps to the exisiting vtables in the code. This is done by taking the function addresses in the vtables and encrypting them, then the encrypted address is placed in some hardcoded shellcode that is used to decrypt and call the address. Each encrypted address/shellcode blob is then called from the code creating some simple obfuscated "redirect" to the actual vtable function. 

![](https://i.imgur.com/6w1xoGP.png)


#### String Decryption
Stolen from [@c3rb3ru5d3d53c](https://gist.github.com/c3rb3ru5d3d53c/4f2c984d81ef64e5f133e37726619c64) with <3

In [67]:
import struct

def decrypt_str(data):
    result = b''
    if (len(data) % 4)!= 0: return None
    for i in range(0, len(data), 4):
        dword = struct.unpack('<I', data[i:i+4])[0]
        dword = (~(dword ^ 0x3B029FD4) & 0xffffffff)
        result += dword.to_bytes(4, byteorder='little')
    return result

dd = struct.pack('<I',0x397F36FB)
dd += struct.pack('<I',0x0D5339D3E)
dd += struct.pack('<I',0x0A4FFA480)
dd += struct.pack('<I',0x9788FEA7)

dd = struct.pack('<I',0x0C4B96067)
dd += struct.pack('<I',0x0C4AD606A)
dd += struct.pack('<I',0x0C4D26011)
dd += struct.pack('<I',0x0C48F6004)
dd += struct.pack('<I',0x0C4926044)
dd += struct.pack('<I',0x0C4B9605F)
dd += struct.pack('<I',0x0C4B86078)
dd += struct.pack('<I',0x0C4FD602B)
decrypt_str(dd)

b'L\x00D\x00A\x00P\x00:\x00/\x00/\x00r\x00o\x00o\x00t\x00D\x00S\x00E\x00\x00\x00\x00\x00'

In [37]:
import re
import struct

# C7 00 (?? ?? ?? ??) 
# C7 40 ?? (?? ?? ?? ??)

test_data = unhex('8d8520ffffffc7006760b9c4c740046a60adc4c740081160d2c4c7400c0460bec4c740106560c0c4c740147b6092c4c74018476094c4c7401c486094c4c740204e608ec4c740240760bec4c740286560c0c4c7402c786084c4c74030586089c4c740344e6090c4c740380760d8c4c7403c5860fdc48b15785140006a1050')

#egg = rb"(?P<bytes>(?P<opcode_byte>.)(?:.){1,2}(?P<imm_data>.){4})"

out = b''
egg = br'\xc7\x00(....)(?P<raw>(?:\xc7\x40.....){1,})'

for match in re.finditer(egg, test_data):
    out += match.group(1)
    g2 = match.group(2)
    for s in g2.split(b'\xc7\x40'):
        out += s[-4:]
        
decrypt_str(out)



b'L\x00D\x00A\x00P\x00:\x00/\x00/\x00C\x00N\x00=\x00P\x00o\x00l\x00i\x00c\x00i\x00e\x00s\x00,\x00C\x00N\x00=\x00S\x00y\x00s\x00t\x00e\x00m\x00,\x00%\x00s\x00\x00\x00'

In [68]:
import uuid

file_data = open('/tmp/d641ad955ef4cff5f0239072b3990d47e17b9840e07fd5feea93c372147313c5_pe.bin', 'rb').read()

#if all(not character for character in b"A\x00B\x00C\x00D\x00E\x00F\x00G\x00\x00"[1::2]):
#    print("not guid")

# output: bytes = b""
# for match in re.finditer(rb"(?P<opcode>[\xC7])(?:[\x00-\x03,\x06-\x07]|[\x40-\x43,\x46-\x47,].)(?P<imm_data>.{4})", b"\xC7\x07sihT\xC7\x40\x20 si \xC7\x40\x20ts a\xC7\x40\x20gnir"):
#     output += match.group("imm_data")[::-1]
# print(output)  # decrypt_string here
# >>> b'This is a string'

# The registers used for the stack string are 
# 0 1 2 3 6 7 

def get_strings(file_data):
    out = {}
    egg = br'\xc7[\x00-\x07](....)(?P<raw>(?:\xc7[\x40-\x47].....){1,})'
    for match in re.finditer(egg, file_data):
        data_offset = match.start()
        tmp_data = b''
        tmp_data += match.group(1)
        g2 = match.group(2)
        split_key = g2[:2]
        for s in g2.split(split_key):
            tmp_data += s[-4:]
        ptxt_data = decrypt_str(tmp_data)
        if ptxt_data[-2:] == b'\x00\x00':
            ptxt_data = ptxt_data.decode('utf-16le')
        elif len(ptxt_data) == 16:
            if any(not c for c in ptxt_data[::2]):
                ptxt_data = str(uuid.UUID(bytes_le=ptxt_data))
        if type(ptxt_data) == str:
            ptxt_data = ptxt_data.replace('\x00','')
        else:
            ptxt_data = ptxt_data.replace(b'\x00',b'')
            try:
                ptxt_data = ptxt_data.decode('utf-8')
            except:
                continue
        out[data_offset] = ptxt_data
    return out
        
get_strings(file_data)
    

{2962: '\\*.dll',
 3152: 'drv',
 7065: 'ea502723-a23d-11d1-a7d3-0000f87571e3',
 7107: 'ea502722-a23d-11d1-a7d3-0000f87571e3',
 7232: 'LDAP://%s',
 7804: '/C DEL /F /Q ',
 7900: ' >> NUL',
 7991: '\\cmd.exe',
 8348: 'LDAP://rootDSE',
 8459: 'defaultNamingContext',
 8596: 'LDAP://CN=Policies,CN=System,%s',
 8817: 'displayName',
 9023: 'distinguishedName',
 9801: 'File',
 9898: 'Preferences',
 10193: '牦浯慐桴∽',
 10661: '%s_IPC$',
 10812: '\\\\.\\pipe\\{%08X-%04X-%04X-%02X%02X-%02X%02X%02X%02X%02X%02X}'}

#### IDA Python Script to Label Strings


```python

def set_hexrays_comment(address, text):
    '''
    set comment in decompiled code
    '''
    cfunc = idaapi.decompile(address)
    tl = idaapi.treeloc_t()
    tl.ea = address
    tl.itp = idaapi.ITP_SEMI
    cfunc.set_user_cmt(tl, text)
    cfunc.save_user_cmts() 


def set_comment(address, text):
    ## Set in dissassembly
    idc.set_cmt(address, text,0)
    ## Set in decompiled data
    set_hexrays_comment(address, text)

    
string_map = {2962: '\\*.dll',
 3152: 'drv',
 7065: 'ea502723-a23d-11d1-a7d3-0000f87571e3',
 7107: 'ea502722-a23d-11d1-a7d3-0000f87571e3',
 7232: 'LDAP://%s',
 7804: '/C DEL /F /Q ',
 7900: ' >> NUL',
 7991: '\\cmd.exe',
 8348: 'LDAP://rootDSE',
 8459: 'defaultNamingContext',
 8596: 'LDAP://CN=Policies,CN=System,%s',
 8817: 'displayName',
 9023: 'distinguishedName',
 9801: 'File',
 9898: 'Preferences',
 10193: '牦浯慐桴∽',
 10661: '%s_IPC$',
 10812: '\\\\.\\pipe\\{%08X-%04X-%04X-%02X%02X-%02X%02X%02X%02X%02X%02X}'}
 
for offset,string_value in string_map.items():    
    va = idaapi.get_fileregion_ea(offset)
    set_comment(va, string_value)
    
```


### Functionality -- STILL TO BE COMPLETED

- The binary registers itself as a service, then creates a named pipe which will recieve 4 bytes to be be converted into a DWORD time value and sent to a thread. The thread's sole purpose is to sleep for the DWORD amount of time. The threads are also hidden from the debugger. It is not clear why this functionality exists, maybe as some sort of time delay?

  - The named pipe name is generated using the MD5 of the string `\\0\<computer_name>.<dc_name>\_IPC$` which is then converted into a "GUID" format following this pattern `\\.\pipe\{%08X-%04X-%04X-%02X%02X-%02X%02X%02X%02X%02X%02X}`


- The binary scans AD via the COM interface and *I think* accesses the ransom binary remotely from the AD server
- The Restart Manager is used to collect a list of services and processes that block the ransomware execution which are then terminated 
- I don't know why they don't launch the ransomware?
- The ransomware is then overwritten with garbage and deleted 




#### Tools
- [Welcome to MagNumDB](https://www.magnumdb.com/) is useful for looking up COM GUIDs

#### References
- [Restart manager](https://docs.microsoft.com/en-us/windows/win32/rstmgr/about-restart-manager)
